In [1]:
    
#import csv
#import time
#import requests
#from bs4 import BeautifulSoup
#from pattern import ngrams

#Base_url = "http://www.moneycontrol.com"

## Build a dictionary of companies and their abbreviated names 
#companies = {'cadilahealthcare':'CHC','piramalenterprises':'PH05',
 #            'glenmarkpharma':'GP08','glaxosmithklinepharmaceuticals':'GSK',
  #           'sunpharmaceuticalindustries':'SPI','lupinlaboratories':'LL',
   #          'cipla':'C','aurobindopharma':'AP',
    #         'drreddyslaboratories':'DRL','divislaboratories':'DL03'}
             
## Create a list of the news section urls of the respective companies 
##url_list = ['https://www.moneycontrol.com/company-article/{}/news/{}#{}'.format(k,v,v) for k,v in companies.iteritems()]

#url_list = ['https://www.moneycontrol.com/company-article/{}/news/{}#{}'.format(k,v,v) for k,v in companies.items()]

#print(url_list)

['https://www.moneycontrol.com/company-article/glenmarkpharma/news/GP08#GP08', 'https://www.moneycontrol.com/company-article/cadilahealthcare/news/CHC#CHC', 'https://www.moneycontrol.com/company-article/lupinlaboratories/news/LL#LL', 'https://www.moneycontrol.com/company-article/drreddyslaboratories/news/DRL#DRL', 'https://www.moneycontrol.com/company-article/piramalenterprises/news/PH05#PH05', 'https://www.moneycontrol.com/company-article/cipla/news/C#C', 'https://www.moneycontrol.com/company-article/divislaboratories/news/DL03#DL03', 'https://www.moneycontrol.com/company-article/glaxosmithklinepharmaceuticals/news/GSK#GSK', 'https://www.moneycontrol.com/company-article/sunpharmaceuticalindustries/news/SPI#SPI', 'https://www.moneycontrol.com/company-article/aurobindopharma/news/AP#AP']


In [9]:
import textblob   
import csv
import time
import requests
from bs4 import BeautifulSoup
#from pattern import ngrams

Base_url = "https://markets.businessinsider.com"

# Build a dictionary of companies and their abbreviated names 
companies = {'vlo','total','delek_energy_systems','hfc','mro','psx','tso'}
             
# Create a list of the news section urls of the respective companies 
#url_list = ['https://www.moneycontrol.com/company-article/{}/news/{}#{}'.format(k,v,v) for k,v in companies.iteritems()]

url_list = ['https://markets.businessinsider.com/news/'+k for k in companies]

print(url_list)

['https://markets.businessinsider.com/news/total', 'https://markets.businessinsider.com/news/delek_energy_systems', 'https://markets.businessinsider.com/news/hfc', 'https://markets.businessinsider.com/news/psx', 'https://markets.businessinsider.com/news/tso', 'https://markets.businessinsider.com/news/mro', 'https://markets.businessinsider.com/news/vlo']


In [10]:
# Create an empty list which will contain the selected news articles 
List_of_links = []   

# Extract the relevant news articles weblinks from the news section of selected companies
for urls in url_list:
    html = requests.get(urls)
    soup = BeautifulSoup(html.text,'html.parser') # Create a BeautifulSoup object 

   # Retrieve a list of all the links and the titles for the respective links
   #word1,word2,word3 = "Results","USA","USFDA"
 
   #sub_links = soup.find_all('a', class_='arial11_summ')
    sub_links = soup.find_all('a', class_='news-link')
    for links in sub_links:
        sp = BeautifulSoup(str(links),'html.parser')  # first convert into a string
        tag = sp.a
      #if word1 in tag['title'] or word2 in tag['title'] or word3 in tag['title']:
          #category_links = Base_url + tag["href"]
        category_links = Base_url 
        List_of_links.append(category_links)
        time.sleep(3)

In [29]:
import sys
#reload(sys)
#sys.setdefaultencoding('utf-8')

AttributeError: module 'sys' has no attribute 'setdefaultencoding'

In [33]:
# Remove the duplicate news articles based on News Title
unique_links = list(set(List_of_links))
for q in unique_links: print(q)

# Create a dictionary of positive/negative words related to the Pharma Sector
reader = csv.reader(open('dict.csv', 'r'))

#name=school_name.encode('utf-8'), street=row[9].encode('utf-8'), city=row[10].encode('utf-8'), state=row[11].encode('utf-8'), zip5=row[12], zip4=row[13],county=row[25].encode('utf-8'), lat=row[22], lng=row[23])



oil_dict = dict((rows[0],rows[1]) for rows in reader)




# Creating an empty list which will be filled later with news article links, and Polarity values (pos/neg)
df =[]

print(df)

https://markets.businessinsider.com


UnicodeDecodeError: 'ascii' codec can't decode byte 0xe6 in position 1181: ordinal not in range(128)

In [ ]:
# Open the choosen news articles and extract the main text  
for selected_links in unique_links:
    results_url = selected_links 
   #print results_url
   

    results = requests.get(results_url)
    results_text = BeautifulSoup(results.text)
    extract_text = results_text.find(class_='article_box')
    final_text = extract_text.get_text()
    
#try:
#    final_text = extract_text.get_text()
#except Exception as a:
    #final_text =''
 #   final_text.append('')
   # pass
            

In [ ]:
final_text